In [1]:
import pandas as pd
import re

load data

In [54]:
rent_csv_file = './data/renttherunway_final_data.csv' 
rent_data = pd.read_csv(rent_csv_file)

In [55]:
rent_data.head(5)

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10.0,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"


Convert weight and height

In [56]:
def height_to_cm(height):
    if not isinstance(height, str): 
        return None 
    match = re.match(r"(\d+)' (\d+)\"", height)
    if match:
        feet = int(match.group(1)) 
        inches = int(match.group(2)) 
        return feet * 30.48 + inches * 2.54
    else:
        return None

In [57]:
def height_to_cm_ft_in(height):
    if not isinstance(height, str):
        return None
    match = re.match(r"(\d+)ft (\d+)in", height)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        return feet * 30.48 + inches * 2.54
    elif 'ft' in height:
        feet = int(height.split('ft')[0])
        return feet * 30.48
    else:
        return None

In [58]:
rent_data['height_cm'] = rent_data['height'].apply(height_to_cm)
rent_data['weight_kg'] = rent_data['weight'].str.replace('lbs', '').astype(float) * 0.453592
rent_data = rent_data.drop(columns = ['weight', 'height'])

In [59]:
rent_data

,fit,user_id,bust size,item_id,rating,rented for,review_text,body type,review_summary,category,size,age,review_date,height_cm,weight_kg
0,fit,420272,34d,2260466,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,14,28.0,"April 20, 2016",172.72,62.142104
1,fit,273551,34b,153475,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,12,36.0,"June 18, 2013",167.64,59.874144
2,fit,360448,NaN,1063761,10.0,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,4,116.0,"December 14, 2015",162.56,NaN
3,fit,909926,34c,126335,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,8,34.0,"February 12, 2014",165.10,61.234920
4,fit,151944,34b,616682,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,12,27.0,"September 26, 2016",175.26,65.770840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,34dd,2252812,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,8,42.0,"May 18, 2016",175.26,63.502880
192540,fit,118398,32c,682043,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,4,29.0,"September 30, 2016",154.94,45.359200
192541,fit,47002,36a,683251,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,8,31.0,"March 4, 2016",172.72,61.234920
192542,fit,961120,36c,126335,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,16,31.0,"November 25, 2015",167.64,74.842680


review_text， review_summary may also need drop, age, review_date, rating should not be useful.

rented for? not easy to use can be droped. 

category has 66 different kinds. Hard to use. 

'bust size' has 2w nan value. drop


In [60]:
rent_data_drop = rent_data.drop(columns = ['review_text', 'age', 'review_date','rented for','rating', 'bust size'])

In [61]:
rent_data_drop

,fit,user_id,item_id,body type,review_summary,category,size,height_cm,weight_kg
0,fit,420272,2260466,hourglass,So many compliments!,romper,14,172.72,62.142104
1,fit,273551,153475,straight & narrow,I felt so glamourous!!!,gown,12,167.64,59.874144
2,fit,360448,1063761,NaN,It was a great time to celebrate the (almost) ...,sheath,4,162.56,NaN
3,fit,909926,126335,pear,Dress arrived on time and in perfect condition.,dress,8,165.10,61.234920
4,fit,151944,616682,athletic,Was in love with this dress !!!,gown,12,175.26,65.770840
...,...,...,...,...,...,...,...,...,...
192539,fit,66386,2252812,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,8,175.26,63.502880
192540,fit,118398,682043,petite,LOVE it!,dress,4,154.94,45.359200
192541,fit,47002,683251,straight & narrow,"Loud patterning, flattering fit",dress,8,172.72,61.234920
192542,fit,961120,126335,pear,loved this dress it was comfortable and photog...,dress,16,167.64,74.842680


If the data missing 'body type' and 'weight_kg', we count them useless.  --> drop (around 5k) 2.5%

there is around 300 nan in review_summary, dropped

In [62]:
missing_bust_and_body = rent_data[rent_data['body type'].isna()& rent_data['weight_kg'].isna()]

In [63]:
rent_data_cleaned = rent_data_drop.dropna(subset=['body type', 'weight_kg'], how='all')

In [64]:
rent_data_cleaned = rent_data_cleaned.dropna(subset=['review_summary'])

In [65]:
rent_data_cleaned

,fit,user_id,item_id,body type,review_summary,category,size,height_cm,weight_kg
0,fit,420272,2260466,hourglass,So many compliments!,romper,14,172.72,62.142104
1,fit,273551,153475,straight & narrow,I felt so glamourous!!!,gown,12,167.64,59.874144
3,fit,909926,126335,pear,Dress arrived on time and in perfect condition.,dress,8,165.10,61.234920
4,fit,151944,616682,athletic,Was in love with this dress !!!,gown,12,175.26,65.770840
5,fit,734848,364092,athletic,Traditional with a touch a sass,dress,8,172.72,62.595696
...,...,...,...,...,...,...,...,...,...
192539,fit,66386,2252812,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,8,175.26,63.502880
192540,fit,118398,682043,petite,LOVE it!,dress,4,154.94,45.359200
192541,fit,47002,683251,straight & narrow,"Loud patterning, flattering fit",dress,8,172.72,61.234920
192542,fit,961120,126335,pear,loved this dress it was comfortable and photog...,dress,16,167.64,74.842680


In [19]:
rent_data_cleaned['love_count'].value_counts(dropna=False)

love_count
0    164497
1     20660
3       603
2       470
4        13
5         7
Name: count, dtype: int64

In [66]:
rent_data_cleaned['love_count'] = rent_data_cleaned['review_summary'].str.count('(?i)love[^\w]*')

In [67]:
rent_data_cleaned

,fit,user_id,item_id,body type,review_summary,category,size,height_cm,weight_kg,love_count
0,fit,420272,2260466,hourglass,So many compliments!,romper,14,172.72,62.142104,0
1,fit,273551,153475,straight & narrow,I felt so glamourous!!!,gown,12,167.64,59.874144,0
3,fit,909926,126335,pear,Dress arrived on time and in perfect condition.,dress,8,165.10,61.234920,0
4,fit,151944,616682,athletic,Was in love with this dress !!!,gown,12,175.26,65.770840,1
5,fit,734848,364092,athletic,Traditional with a touch a sass,dress,8,172.72,62.595696,0
...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,2252812,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,8,175.26,63.502880,1
192540,fit,118398,682043,petite,LOVE it!,dress,4,154.94,45.359200,1
192541,fit,47002,683251,straight & narrow,"Loud patterning, flattering fit",dress,8,172.72,61.234920,0
192542,fit,961120,126335,pear,loved this dress it was comfortable and photog...,dress,16,167.64,74.842680,1


In [68]:
rent_data_cleaned['height_m'] = rent_data_cleaned['height_cm'] / 100

# Calculate BMI
rent_data_cleaned['BMI'] = rent_data_cleaned['weight_kg'] / (rent_data_cleaned['height_m'] ** 2)

# Drop the intermediate column if desired
rent_data_cleaned.drop(columns=['height_m'], inplace=True)

In [74]:
rent_data_cleaned[rent_data_cleaned["size"] > 50]

,fit,user_id,item_id,body type,review_summary,category,size,height_cm,weight_kg,love_count,BMI,BMI_Range
110,fit,603920,191514,full bust,I absolutely loved this dress!,dress,51,167.64,113.398000,1,40.350609,Obese Class III
143,small,281878,1920477,full bust,"Adorable, sweet cocktail dress",dress,51,154.94,97.522280,0,40.623444,Obese Class III
245,fit,932954,1109803,full bust,"Needed a ""stunner"" dress for my formal company...",gown,58,172.72,NaN,0,NaN,Obese Class III
246,small,893699,1889597,apple,Great for a wedding!,sheath,51,167.64,99.790240,0,35.508536,Obese Class III
369,fit,656407,387552,full bust,It fit perfect and was so easy to return!,gown,57,175.26,112.490816,0,36.622792,Obese Class III
...,...,...,...,...,...,...,...,...,...,...,...,...
192161,large,222319,1868145,pear,Soft and comfortable,dress,51,167.64,130.634496,0,46.483901,Obese Class III
192312,small,134614,157448,pear,Everyone said I looked fabulous. No surprise ...,sheath,57,165.10,115.665960,0,42.433751,Obese Class III
192338,fit,306019,126335,apple,I was aiming for a vintage movie star look for...,dress,51,162.56,98.429464,0,37.247552,Obese Class III
192414,fit,363586,1542250,full bust,"Gorgeous detail, impeccable tailoring, luxurio...",dress,51,160.02,NaN,0,NaN,Obese Class III


In [80]:
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi <= 23.9:
        return "Normal weight"
    elif 24 <= bmi <= 27.9:
        return "Overweight"
    elif 28 <= bmi <= 30:
        return "Obese Class I"
    elif 30 < bmi <= 35:
        return "Obese Class II"
    else:
        return "Obese Class III"

rent_data_cleaned['BMI_Range'] = rent_data_cleaned['BMI'].apply(categorize_bmi)


# rent_data_cleaned[rent_data_cleaned["category"] == "pants"]["size"].max()


In [71]:

avg_size_by_body_type = df.groupby('body type')['size'].mean().reset_index()
avg_size_by_body_type.columns = ['body type', 'average_size']

review_count_by_item = df.groupby('item_id').size().reset_index(name='review_count')


item_body_mapping = df[['item_id', 'body type']].drop_duplicates()

pd.DataFrame(avg_size_by_body_type)

,body type,average_size
0,apple,21.274240
1,athletic,10.634187
2,full bust,17.856104
3,hourglass,14.403073
4,pear,14.037520
5,petite,5.525930
6,straight & narrow,7.319496


In [72]:
pd.DataFrame(review_count_by_item)


,item_id,review_count
0,123373,673
1,123793,1670
2,124204,790
3,124553,481
4,125424,448
...,...,...
5833,2963850,11
5834,2964470,3
5835,2965009,4
5836,2965924,3
